In [1]:
import multiversx_utils as mu
import json
import pandas as pd

In [2]:
collections = ['CRMYTH-546419', 'CRWEAPONS-e5ab49', 'GSPACEAPE-08bc2b', 'CEA-2d29f9', 'CRCHAMPS-d0265d', 'CRHEROES-9edff2', 'EAPES-8f3c1f']
info = False
nfts_json = False
nfts_csv = False
txs = False

In [ ]:
params = {
    'CRMYTH-546419': {
        'sleep_time': 0.6,
        'whitelist': ['https://metadata.cantinaroyale.io/dynamic/']
    },
    'CRWEAPONS-e5ab49': {
        'sleep_time': 0.6,
        'whitelist': ['https://metadata.cantinaroyale.io/dynamic/']
    },
    'GSPACEAPE-08bc2b': {
        'sleep_time': 0.2,
        'whitelist': ['https://metadata.verko.io/dynamic/']
    },
    'CEA-2d29f9': {
        'sleep_time': 0.2,
        'whitelist': ['https://metadata.verko.io/dynamic/']
    },
    'CRCHAMPS-d0265d': {
        'sleep_time': 0.6,
        'whitelist': ['https://metadata.cantinaroyale.io/dynamic/', 'https://metadata.cantinaroyale.io/metadata/']
    },
    'CRHEROES-9edff2': {
        'sleep_time': 0.6,
        'whitelist': ['https://metadata.cantinaroyale.io/dynamic/', 'https://metadata.cantinaroyale.io/metadata/']
    },
    'EAPES-8f3c1f': {
        'sleep_time': 0,
        'whitelist': []
    }
}

In [ ]:
def get_collection_info(collection_name):
    collection_info = mu.get_collection_info(collection_name)
    with open(f'data/{collection_name}_info.json', 'w') as f:
        json.dump(collection_info, f, indent=4)

def get_collection_nfts_json(collection_name):
    collection_nfts = mu.get_collection_nfts(collection_name)
    sleep_time = params[collection_name]['sleep_time']
    whitelist = params[collection_name]['whitelist']
    collection_offchain_data = mu.get_collection_offchain_data(collection_nfts, sleep_time=sleep_time, whitelist=whitelist)
    for identifier in collection_nfts:
        collection_nfts[identifier]['offchain_data'] = collection_offchain_data[identifier]
    with open(f'data/{collection_name}_nfts.json', 'w') as f:
        json.dump(collection_nfts, f, indent=4)

def get_collection_nfts_csv(collection_name):
    with open(f'data/{collection_name}_nfts.json', 'r') as f:
        collection_nfts = json.load(f)
    collection = {}
    for nft in collection_nfts.values():
        nft_data = mu.parse_nft_data(nft)
        for key, value in nft_data.items():
            if key not in collection:
                collection[key] = []
            collection[key].append(value)
    pd.DataFrame(collection).to_csv(f'{collection_name}_nfts.csv', index=False)

def get_collection_txs(collection_name):
    collection_transactions = mu.get_collection_txs(collection_name)
    with open(f'data/{collection_name}_txs.json', 'w') as f:
        json.dump(collection_transactions, f, indent=4)

In [ ]:
for collection_name in collections:
    if info:
        get_collection_info(collection_name)
    if nfts_json:
        get_collection_nfts_json(collection_name)
    if nfts_csv:
        get_collection_nfts_csv(collection_name)
    if txs:
        get_collection_txs(collection_name)